In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
data=pd.read_csv("Electric_Vehicle_Data.csv")
data

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Base MSRP,Legislative District,DOL Vehicle ID,Vehicle Location,Electric Utility,2020 Census Tract
0,WAUTPBFF4H,King,Seattle,WA,98126.0,2017,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,34.0,235085336,POINT (-122.374105 47.54468),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),5.303301e+10
1,WAUUPBFF2J,Thurston,Olympia,WA,98502.0,2018,AUDI,A3,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,16,0,22.0,237896795,POINT (-122.943445 47.059252),PUGET SOUND ENERGY INC,5.306701e+10
2,5YJSA1E22H,Thurston,Lacey,WA,98516.0,2017,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,210,0,22.0,154498865,POINT (-122.78083 47.083975),PUGET SOUND ENERGY INC,5.306701e+10
3,1C4JJXP62M,Thurston,Tenino,WA,98589.0,2021,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,25,0,20.0,154525493,POINT (-122.85403 46.856085),PUGET SOUND ENERGY INC,5.306701e+10
4,5YJ3E1EC9L,Yakima,Yakima,WA,98902.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,14.0,225996361,POINT (-120.524012 46.5973939),PACIFICORP,5.307700e+10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181453,50EA1TEA7P,Douglas,East Wenatchee,WA,98802.0,2023,LUCID,AIR,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,12.0,244207316,POINT (-120.28674 47.4176),PUD NO 1 OF DOUGLAS COUNTY,5.301795e+10
181454,1C4JJXP60N,Spokane,Spokane Valley,WA,99206.0,2022,JEEP,WRANGLER,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,21,0,4.0,207180774,POINT (-117.24549 47.6534),BONNEVILLE POWER ADMINISTRATION||AVISTA CORP||...,5.306301e+10
181455,5YJ3E1EA0M,King,Enumclaw,WA,98022.0,2021,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,0,0,31.0,161934202,POINT (-121.98953 47.20347),PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),5.303303e+10
181456,5YJ3E1EC8L,Clark,Vancouver,WA,98682.0,2020,TESLA,MODEL 3,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,308,0,18.0,100859650,POINT (-122.5286031 45.6686601),BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF C...,5.301104e+10


In [5]:
data.columns

Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')

In [23]:
class Generalizer:
  def __init__(self, df) -> None:
    self.data = df.copy(deep=True)
  
  def fill_zero_with_nan(self, target_column):
    self.data.loc[self.data[target_column] == 0, target_column] = np.nan

  def generalise_continous_to_categorical(self, target_column, no_of_categories, labels=None):
    bins = pd.qcut(self.data[target_column], no_of_categories, retbins=True)[1]
    
    if labels == None:
      labels = [f"{int(bins[i])}-{int(bins[i+1])}" for i in range(len(bins)-1)]
    else:
      labels = [f"{labels[i]} ({int(bins[i])}-{int(bins[i+1])})" for i in range(len(bins)-1)]

    self.data[target_column] = pd.cut(self.data[target_column], bins=bins, include_lowest=True, labels=labels)

  def generalize_year(self):
    self.generalise_continous_to_categorical('Model Year', no_of_categories=3)
  
  def generalize_range(self):
    self.fill_zero_with_nan('Electric Range')
    self.generalise_continous_to_categorical('Electric Range', no_of_categories=3, labels=['Short Range', 'Medium Range', 'Long Range'] ) 

  def generalize_Electric_Utility(self):
    self.data.dropna(subset=['Electric Utility'], inplace=True)
    self.data['Electric Utility'] = self.data['Electric Utility'].apply(lambda x: x.split('||')[0])
  
  def generalize_ev_type(self):
    mp = {
      'Battery Electric Vehicle (BEV)': 'BEV',
      'Plug-in Hybrid Electric Vehicle (PHEV)': 'PHEV'
    }
    self.data['Electric Vehicle Type'] = self.data['Electric Vehicle Type'].apply(lambda x: mp[x])

  def generalize_cafv_eligibility(self):
    mp = {
      'Not eligible due to low battery range': 'NO',
      'Eligibility unknown as battery range has not been researched': 'NO',
      'Clean Alternative Fuel Vehicle Eligible': 'YES'
    }
    self.data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'] = self.data['Clean Alternative Fuel Vehicle (CAFV) Eligibility'].apply(lambda x: mp[x])

  def remove_unused_fields(self):
    unnecessary_fields = ['VIN (1-10)', 'City', 'State', 'Model', 'Postal Code', 'Base MSRP', 'DOL Vehicle ID', 'Vehicle Location', 'Legislative District', '2020 Census Tract']
    self.data.drop(labels=unnecessary_fields, inplace=True, axis=1)
    
  def generalize(self):
    self.generalize_year()
    self.generalize_range()
    self.generalize_Electric_Utility()
    self.generalize_ev_type()
    self.generalize_cafv_eligibility()

  def get_count_table(self, df):
    group_by_columns = df.columns.tolist()
    
    summary_table = df.groupby(df.columns.tolist()).size().reset_index(name='count')

    summary_table.columns = [*group_by_columns, 'Count']

    summary_table = summary_table.sort_values(group_by_columns).reset_index(drop=True)

    return summary_table

  def get_generalised(self):
    return self.get_count_table(self.data)

In [24]:
gen = Generalizer(data)

# print(gen.get_generalised()['Base MSRP'].unique())

gen.remove_unused_fields()
gen.generalize()
temp = gen.get_generalised()

temp

,County,Model Year,Make,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,Electric Range,Electric Utility,Count
0,Ada,1997-2020,ALFA ROMEO,BEV,NO,Short Range (6-37),AVISTA CORP,0
1,Ada,1997-2020,ALFA ROMEO,BEV,NO,Short Range (6-37),BONNEVILLE POWER ADMINISTRATION,0
2,Ada,1997-2020,ALFA ROMEO,BEV,NO,Short Range (6-37),CITY OF BLAINE - (WA),0
3,Ada,1997-2020,ALFA ROMEO,BEV,NO,Short Range (6-37),CITY OF CHENEY - (WA),0
4,Ada,1997-2020,ALFA ROMEO,BEV,NO,Short Range (6-37),CITY OF CHEWELAH,0
...,...,...,...,...,...,...,...,...
6114235,Yuba,2023-2024,WHEEGO ELECTRIC CARS,PHEV,YES,Long Range (208-337),PUD NO 1 OF OKANOGAN COUNTY,0
6114236,Yuba,2023-2024,WHEEGO ELECTRIC CARS,PHEV,YES,Long Range (208-337),PUD NO 1 OF PEND OREILLE COUNTY,0
6114237,Yuba,2023-2024,WHEEGO ELECTRIC CARS,PHEV,YES,Long Range (208-337),PUD NO 1 OF WHATCOM COUNTY,0
6114238,Yuba,2023-2024,WHEEGO ELECTRIC CARS,PHEV,YES,Long Range (208-337),PUD NO 2 OF GRANT COUNTY,0


In [24]:
def get_pivot_table(df, index_col, other_cols):
  pivot_table = pd.pivot_table(df, index=index_col, columns=other_cols, aggfunc='size', fill_value=0)
  pivot_table['Total'] = pivot_table.sum(axis=1)
  pivot_table.loc['Total'] = pivot_table.sum(axis=0)
  return pivot_table

In [25]:
evType_vs_range = get_pivot_table(temp, index_col='Electric Vehicle Type', other_cols='Electric Range')

evType_vs_range

Electric Range,Short Range (6-37),Medium Range (37-208),Long Range (208-337),Total
Electric Vehicle Type,,,,
BEV,9,19149,28082,47240
PHEV,29143,10342,0,39485
Total,29152,29491,28082,86725


In [26]:
evType_vs_utility = get_pivot_table(temp, index_col='Electric Vehicle Type', other_cols='Electric Utility')

evType_vs_utility

Electric Utility,AVISTA CORP,BONNEVILLE POWER ADMINISTRATION,CITY OF BLAINE - (WA),CITY OF CHENEY - (WA),CITY OF CHEWELAH,CITY OF SEATTLE - (WA),CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),CITY OF SUMAS - (WA),CITY OF TACOMA - (WA),MODERN ELECTRIC WATER COMPANY,...,PACIFICORP,PORTLAND GENERAL ELECTRIC CO,PUD NO 1 OF CHELAN COUNTY,PUD NO 1 OF DOUGLAS COUNTY,PUD NO 1 OF OKANOGAN COUNTY,PUD NO 1 OF PEND OREILLE COUNTY,PUD NO 1 OF WHATCOM COUNTY,PUD NO 2 OF GRANT COUNTY,PUGET SOUND ENERGY INC,Total
Electric Vehicle Type,,,,,,,,,,,,,,,,,,,,,
BEV,269,25350,153,35,5,1,25633,4,232,827,...,930,4,856,280,68,36,25,415,86303,141970
PHEV,140,9653,40,10,4,0,6379,3,48,327,...,411,1,222,87,37,20,16,170,21672,39485
Total,409,35003,193,45,9,1,32012,7,280,1154,...,1341,5,1078,367,105,56,41,585,107975,181455


In [27]:
evType_vs_cafv = get_pivot_table(temp, index_col='Electric Vehicle Type', other_cols='Clean Alternative Fuel Vehicle (CAFV) Eligibility')

evType_vs_cafv

Clean Alternative Fuel Vehicle (CAFV) Eligibility,NO,YES,Total
Electric Vehicle Type,,,
BEV,94739,47231,141970
PHEV,19903,19582,39485
Total,114642,66813,181455
